# OSCAR, a Tech Preview (Part I)

A tiny example of using polymake and GAP combined.

## The Setup

The configuration can be cloned via copying the <i>Project.toml</i> file and using <i>instantiate</i> in the REPL mode.

In [1]:
versioninfo()

using Pkg
Pkg.status()

Julia Version 1.2.0
Commit c6da87ff4b (2019-08-20 00:03 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i7-7600U CPU @ 2.80GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)
    Status `~/julia/oscar/Project.toml`
  [c87230d0] FFMPEG v0.2.4
  [c863536a] GAP v0.2.3
  [f213a82b] HomotopyContinuation v1.3.1
  [7073ff75] IJulia v1.20.2
  [55ecb840] ImplicitPlots v0.1.0 #master (https://github.com/saschatimme/ImplicitPlots.git)
  [739be429] MbedTLS v0.7.0
  [2edaba10] Nemo v0.16.1
  [d720cf60] Polymake v0.2.2
  [bcd08a7b] Singular v0.1.0


## polymake's Functions Are Available

In [2]:
import Polymake

polymake version 3.5
Copyright (c) 1997-2019
Ewgenij Gawrilow, Michael Joswig (TU Berlin)
https://polymake.org

This is free software licensed under GPL; see the source for copying conditions.
There is NO warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In the context of Hilbert's 16th Problem Itenberg and Viro constructed a counter-example to a conjecture of Ragdale, via <i>patchworking</i>.  This is just to show that (almost) all of polymake's functionality is available through Julia.

In [3]:
R = Polymake.Tropical.ragsdale_counterexample();

polymake: used package ppl
  The Parma Polyhedra Library ([[wiki:external_software#PPL]]): A C++ library for convex polyhedra
  and other numerical abstractions.
  http://www.cs.unipr.it/ppl/



In [4]:
Polymake.Tropical.visual(R)

polymake: used package cdd
  cddlib
  Implementation of the double description method of Motzkin et al.
  Copyright by Komei Fukuda.
  http://www-oldurls.inf.ethz.ch/personal/fukudak/cdd_home/



<!--
polymake for joswig
Fri Jan 24 20:50:21 2020
pcom:
-->


 
 
 pcom: 
 
 

 

 
 
 Explode 
 
 Automatic explosion 
 Exploding speed 
 
 

 
 
 Transparency 
 
 
 
 
 Rotation 
 
 x-axis 
 y-axis 
 z-axis 
 Reset 
 

 Rotation speed 
 

 


 
 Display 
 
 
 Labels 
 
 


 
 SVG 
 
 
 Download 
 New tab 
 
 Screenshot 
 
 

 <!-- end of settings -->

polymake: used package threejs
   Three.js is a lightweight cross-browser JavaScript library/API used to create and display animated 3D computer graphics on a Web browser.
   See http://github.com/mrdoob for the source code.



The actual example starts here.  We construct a regular $4$-cube, with $\pm1$-coordinates, but this will not matter here.

In [5]:
C = Polymake.Polytope.cube(4)

type: Polytope<Rational>
description: cube of dimension 4


CONE_AMBIENT_DIM
5

CONE_DIM
5

FACETS
(5) (0 1) (1 1)
(5) (0 1) (1 -1)
(5) (0 1) (2 1)
(5) (0 1) (2 -1)
(5) (0 1) (3 1)
(5) (0 1) (3 -1)
(5) (0 1) (4 1)
(5) (0 1) (4 -1)


AFFINE_HULL


VERTICES_IN_FACETS
{0 2 4 6 8 10 12 14}
{1 3 5 7 9 11 13 15}
{0 1 4 5 8 9 12 13}
{2 3 6 7 10 11 14 15}
{0 1 2 3 8 9 10 11}
{4 5 6 7 12 13 14 15}
{0 1 2 3 4 5 6 7}
{8 9 10 11 12 13 14 15}


BOUNDED
true

We can get the combinatorial automorphism group.  Julia issues a warning to indicate that some data obtained from polymake is opaque and cannot be accessed directly in a Julia-typical way.

In [6]:
G_polymake = Polymake.Group.automorphism_group(C.VERTICES_IN_FACETS)

┌ Warning: The return value contains pm::IncidenceMatrix<pm::NonSymmetric> which has not been wrapped yet;
│ use `@pm Common.convert_to{wrapped_type}(...)` to convert to julia-understandable type.
└ @ Polymake /home/joswig/.julia/packages/Polymake/RJHSX/src/functions.jl:66
polymake: used package bliss
  [[wiki:external_software#bliss]] is a software for computation of automorphism groups of graphs.
  Copyright by Tommi Junttila and Petteri Kaski.
  http://www.tcs.hut.fi/Software/bliss/


type: Group

PERMUTATION_ACTION
type: PermutationAction<Int, Rational>

The group is given as a permutation group, acting on the eight facets.

In [7]:
gens = G_polymake.PERMUTATION_ACTION.GENERATORS

pm::Array<pm::Array<int>>
0 1 2 3 6 7 4 5
0 1 4 5 2 3 6 7
2 3 0 1 4 5 6 7
1 0 2 3 4 5 6 7


Note that polymake is $0$-based, while Julia is $1$-based.

In [8]:
gens[2]

pm::Array<int>
0 1 4 5 2 3 6 7

Generic Julia functions can convert between $0$- and $1$-based containers.  Future Polymake.jl versions will have this built-in.

In [9]:
to_one_based_indexing(n::Number) = n + one(n)
to_zero_based_indexing(n::Number) = (n > zero(n) ? n - one(n) : throw(ArgumentError("Can't use negative index")))

for f in [:to_one_based_indexing, :to_zero_based_indexing]
    @eval begin
        $f(itr) = $f.(itr)
        $f(s::S) where S<:AbstractSet = S($f.(s))
    end
end

Applying this to the generators of the group computed by polymake yields a standard Julia datatype.

In [10]:
to_one_based_indexing(gens)

4-element Array{Array{Int32,1},1}:
 [1, 2, 3, 4, 7, 8, 5, 6]
 [1, 2, 5, 6, 3, 4, 7, 8]
 [3, 4, 1, 2, 5, 6, 7, 8]
 [2, 1, 3, 4, 5, 6, 7, 8]

## Use GAP For Groups

The software of choice to deal with groups is GAP.  Here is a micro-demo, which constructs a symmetric group of degree four from a 4-cycle and a transposition.

In [11]:
import GAP

S4 = GAP.Globals.Group(GAP.@gap [(1,2,3,4), (1,2)])
@show S4
GAP.Globals.Order(S4)

Adding path /home/joswig/.julia/packages/GAP/aZjvF/gap/.libs to DL_LOAD_PATH
 ┌───────┐   GAP 4.11dev-1190-gceb7d49 of today
 │  GAP  │   https://www.gap-system.org
 └───────┘   Architecture: x86_64-pc-linux-gnu-julia64-kv7
 Configuration:  gmp 6.1.2, Julia GC, Julia 1.2.0, readline
 Loading the library and packages ...
 Packages:   GAPDoc 1.6.3, PrimGrp 3.4.0, SmallGrp 1.4.1, TransGrp 2.0.4
 Try '??help' for help. See also '?copyright', '?cite' and '?authors'
S4 = GAP: Group([ (1,2,3,4), (1,2) ])


24

A typical OSCAR-function will provide natural ways of communication between the corner stone systems.  The function is fairly short, the bulk of the code are some straightforward conversions.

In [12]:
function combinatorial_automorphism_group(P)
    G = Polymake.Group.automorphism_group(P.VERTICES_IN_FACETS)
    gens_polymake = G.PERMUTATION_ACTION.GENERATORS # acting on the facets
    gens_julia = Vector{Int64}.(Polymake.to_one_based_indexing(gens_polymake))
    gens_gap = GAP.Globals.PermList.(GAP.julia_to_gap.(gens_julia))
    return GAP.Globals.Group(gens_gap...)
end

combinatorial_automorphism_group (generic function with 1 method)

Recognizing the isomorphism type of the (combinatorial) automorphism group of a polytope is available neither in GAP nor polymake alone.

In [13]:
G = combinatorial_automorphism_group(C)
GAP.Globals.StructureDescription(G)

┌ Warning: The return value contains pm::IncidenceMatrix<pm::NonSymmetric> which has not been wrapped yet;
│ use `@pm Common.convert_to{wrapped_type}(...)` to convert to julia-understandable type.
└ @ Polymake /home/joswig/.julia/packages/Polymake/RJHSX/src/functions.jl:66


GAP: "(C2 x C2 x C2 x C2) : S4"

Some function are implemented independently in both systems.  This can be employed for cross-certification.

In [14]:
@show G_polymake.ORDER
@show GAP.Globals.Order(G)

G_polymake.ORDER = 384
GAP.Globals.Order(G) = 

polymake: used package permlib
  A callable C++ library for permutation computations. 
  Written by Thomas Rehn.
  http://www.math.uni-rostock.de/~rehn/software/permlib.html 



384


384